In [7]:
## import required libraries
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import math
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import mean_squared_error

In [16]:
## bulidng a class to convert time series to images

class MakeImage:

    def __init__(self):

        self._df = None 
        self._size_width = None 
        self._index_of_columns_2_be_batched = None 

    @property
    def df(self):
        
        return self._df
    
    @property
    def size_width(self):
        return self._size_width	

    @property
    def index_of_columns_2_be_batched(self):
        return self._index_of_columns_2_be_batched

    @df.setter
    def df(self, value):
        self._df = value

    @size_width.setter
    def size_width(self, value):
        self._size_width = value 

    @index_of_columns_2_be_batched.setter
    def index_of_columns_2_be_batched(self, value):
        self._index_of_columns_2_be_batched = value

    def add_label(self):
        df = self.df
        df.ix[0,'label'] = 0.0
        df.ix[1:,'label'] = [1.0 if df.ix[i,'load_data'] > df.ix[i-1,'load_data'] else 0.0  for i in range(1,len(df.index))]

    def chunk_features_and_label(self, col_name, col_label):
        df = self.df
        image_width = self.size_width
        nparray_features = np.zeros(shape = (len(df.index) - image_width, image_width))
        nparray_label = np.zeros(shape = (len(df.index) - image_width, 1))
                
        for i in range(len(df.index) - image_width):
            nparray_features[i, :]  = df.ix[i: i + image_width - 1, col_name].as_matrix().reshape(image_width)
            nparray_label[i, :] = df.ix[i + image_width, col_label].reshape(-1,1)
        return  nparray_features, nparray_label

    def features_2_images(self, x):
        size_width = self.size_width
        x_image = np.zeros(shape = (x.shape[0], x.shape[1]*x.shape[1]))

        print(x.shape[0])

        for i in range(x.shape[0]):
            x_temp = x[i, :].copy()
            x_copy = np.copy(x[i,:])
            x_int = np.zeros(shape = (x_copy.shape), dtype = int)
            
            x_copy2 = np.copy(x[i,:])
            x_copy.sort()

            for j in range(np.size(x_copy)):
                for k in range(np.size(x_copy)):
                    if x_copy2[j] == x_copy[k]:
                        x[i, j] = int(k) 
            
            n_values = np.max(x[i,:].astype(int)) + 1
            squared = np.eye(n_values)[x[i,:].astype(int)]
            flatten = squared.flatten()
            x_image[i, :] = flatten
        return x_image

    def convert_2_onehot_encoding(self, vector_of_one_dim):

        nparray_lab_onehot = np.zeros(shape = (vector_of_one_dim.shape[0], 2))
        
        for i in range(vector_of_one_dim.shape[0]):
            n_values = 2

            nparray_lab_onehot[i, :] = np.eye(n_values)[vector_of_one_dim[i,:].astype(int)]
        
        return nparray_lab_onehot

    def number_of_batches(self, x, batch_size):
        return   int(x.shape[0]/batch_size)

    def next_batch(self, x, y, batch_size):
        index_for_batch = self.index_of_columns_2_be_batched

        if len(index_for_batch) >= batch_size:
            selected_index = random.sample(index_for_batch, batch_size)

        else:
            selected_index = random.sample(index_for_batch, len(index_for_batch))
 
        x_batch = [x[i] for i in selected_index]
        y_batch = [y[i] for i in selected_index]

        self.index_of_columns_2_be_batched = [i for i in index_for_batch if i not in selected_index]
        return x_batch, y_batch

In [4]:
def cnn_model():

# building computation graph

    x = tf.placeholder(tf.float32, [None, 1024])   # 32*32
    y = tf.placeholder(tf.float32, [None, 2])
    input_layer_of_load_images = tf.reshape(x, [-1, 32, 32, 1])

    convolute_layer_1 = tf.layers.conv2d(
    inputs = input_layer_of_load_images,
    filters = 32,
    kernel_size = [5, 5],
    padding = "same",
    activation = tf.nn.relu)

    pool_layer_1 = tf.layers.max_pooling2d(
    inputs=convolute_layer_1,
    pool_size=[2, 2],
    strides=2)

    convolute_layer_2 = tf.layers.conv2d(
    inputs=pool_layer_1,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

    pool_layer_2 = tf.layers.max_pooling2d(inputs = convolute_layer_2 , 
    pool_size=[2, 2], 
    strides=2)

    pool_layer_2_flat = tf.reshape(pool_layer_2,
    [-1, 8 * 8 * 64])

    dense_layer_1 = tf.layers.dense(inputs = pool_layer_2_flat,
    units=1024,
    activation=tf.nn.relu)

    dense_layer_2 = tf.layers.dense(inputs = dense_layer_1,
    units = 600,
    activation=tf.nn.relu)

    dropout = tf.layers.dropout(
    inputs = dense_layer_2, rate = 0.4)

    dense_layer_3 = tf.layers.dense(inputs = dropout,
    units=400)

    prediction = tf.layers.dense(dense_layer_3, activation=tf.nn.softmax, units = 2)
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(prediction), reduction_indices=[1]))  

    train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

    number_of_epochs = 15

# creating  session for running computation graph
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(number_of_epochs):
            epoch_loss = 0
            makeImage.index_of_columns_2_be_batched = [i for i in range(0, X_train.shape[0])]

            for _ in range(number_of_batches):
                epoch_x, epoch_y = makeImage.next_batch(X_train, y_train, batch_size)
                _, c = sess.run([train_step, cross_entropy], feed_dict={x: epoch_x, y: epoch_y})

                epoch_loss += c
            print('Epoch', epoch + 1, 'completed out of', number_of_epochs,' with loss:',epoch_loss)

            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

            print('Accuracy in epoch', epoch + 1, 'is:',accuracy.eval({x:X_test, y:y_test}))

In [8]:
# Load Oahu Data
df = pd.read_pickle("df_oahu.pkl")

#Set target and input variables 
target_station = 'DHHL_3'

## Remove columns with many corrupted or missing values
df.drop(columns=['AP_1', 'AP_7'], inplace=True)

In [ ]:
# Split data
interval = ((df.index >= '2010-06') & (df.index < '2011-06'))
#interval = ((df.index >= '2010-11') & (df.index <= '2010-12'))

(train_df, validation_df, test_df) = split_data(df, interval)
(norm_train_df, norm_validation_df, norm_test_df) = split_data(norm_df, interval)

In [13]:
interval = ((df.index >= '2010-06') & (df.index < '2011-06'))

cnn_df = pd.DataFrame({"load_data": df[target_station].iloc[interval].values})

In [17]:
# create a class and initlize an instance of it
makeImage = MakeImage()
# Setter instance variables
makeImage.df = cnn_df
makeImage.size_width = 32
makeImage.add_label()

/Users/cseveriano/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [19]:
# process load data to be proper for the model
X_not_ready_yet, Y_not_ready_yet = makeImage.chunk_features_and_label('load_data', 'label')

In [23]:
X_ready = makeImage.features_2_images(X_not_ready_yet)
Y_ready = makeImage.convert_2_onehot_encoding(Y_not_ready_yet)

22233


In [24]:
# select 70% of data for training the model and 30% for testing
X_train, X_test, y_train, y_test = train_test_split(X_ready, Y_ready, test_size = 0.3)

# the batch size is selected to be 100, it is possible to adjust it based on problem requirment
batch_size = 100
number_of_batches = makeImage.number_of_batches(X_train, batch_size)

In [25]:
cnn_model()

Epoch 1 completed out of 15  with loss: 89.1873759329
Accuracy in epoch 1 is: 0.726237
Epoch 2 completed out of 15  with loss: 81.4176984727
Accuracy in epoch 2 is: 0.725337
Epoch 3 completed out of 15  with loss: 77.3923944831
Accuracy in epoch 3 is: 0.726237
Epoch 4 completed out of 15  with loss: 74.5954051316
Accuracy in epoch 4 is: 0.727436
Epoch 5 completed out of 15  with loss: 70.1307245195
Accuracy in epoch 5 is: 0.724738
Epoch 6 completed out of 15  with loss: 64.573341459
Accuracy in epoch 6 is: 0.725487
Epoch 7 completed out of 15  with loss: 55.7144174725
Accuracy in epoch 7 is: 0.704048
Epoch 8 completed out of 15  with loss: 45.6330631524
Accuracy in epoch 8 is: 0.698051
Epoch 9 completed out of 15  with loss: 34.9945664108
Accuracy in epoch 9 is: 0.682759
Epoch 10 completed out of 15  with loss: 24.2743060961
Accuracy in epoch 10 is: 0.684708
Epoch 11 completed out of 15  with loss: 17.0319530889
Accuracy in epoch 11 is: 0.695652
Epoch 12 completed out of 15  with loss: